In [15]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [21]:
santacruz_df = pd.read_csv(r"data\unlabelled\santacruz_cleaned.csv")
santacruz_df

,INDEX,YEAR,MN,DT,HR,SLP,MSLP,DBT,WBT,DPT,RH,DD,FFF,RF
0,43003,2010,1,1,0,1008.4,1010.1,21.2,17.4,14.8,67,00,000,0.0
1,43003,2010,1,2,0,1009.5,1011.2,22.0,19.4,17.8,77,00,000,0.0
2,43003,2010,1,3,0,1011.5,1013.2,24.0,21.4,20.0,78,00,000,0.0
3,43003,2010,1,4,0,1012.0,1013.7,22.4,18.6,16.1,68,32,006,0.0
4,43003,2010,1,5,0,1009.3,1011.0,19.4,18.4,17.8,90,00,000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35271,43003,2024,11,26,84,0.0,0.0,19.4,27.4,13.5,69,00,000,0.0
35272,43003,2024,11,27,84,0.0,0.0,22.0,27.4,11.7,52,00,000,0.0
35273,43003,2024,11,28,84,0.0,0.0,19.6,27.4,12.1,62,00,000,0.0
35274,43003,2024,11,29,84,0.0,0.0,22.0,27.4,12.5,55,05,006,0.0


In [22]:
df_normal = santacruz_df.groupby(["MN", "DT"])["DBT"].mean().reset_index()
df_normal.rename(columns={"DBT": "Normal_Temp"}, inplace=True)

# Step 2: Find the Daily Maximum Temperature
df_daily_max = santacruz_df.groupby(["YEAR", "MN", "DT"])["DBT"].max().reset_index()

# Step 3: Merge with Normal Temperature Data
df_daily_max = df_daily_max.merge(df_normal, on=["MN", "DT"], how="left")

# Step 4: Calculate Temperature Deviation
df_daily_max["Temp_Deviation"] = df_daily_max["DBT"] - df_daily_max["Normal_Temp"]

# Step 5: Apply IMD Heatwave Classification Criteria
# Heatwave conditions (IMD): DBT >= 37°C and deviation >= 4.5°C
df_daily_max["Heatwave"] = np.where(
    (df_daily_max["DBT"] >= 37) & (df_daily_max["Temp_Deviation"] >= 4.5), 1, 0
)


In [24]:
df_daily_max

,YEAR,MN,DT,DBT,Normal_Temp,Temp_Deviation,Heatwave
0,2010,1,1,32.6,23.708696,8.891304,0
1,2010,1,2,34.4,23.793548,10.606452,0
2,2010,1,3,31.4,23.696774,7.703226,0
3,2010,1,4,28.6,23.391489,5.208511,0
4,2010,1,5,30.4,23.555789,6.844211,0
...,...,...,...,...,...,...,...
4760,2024,11,26,32.8,27.228889,5.571111,0
4761,2024,11,27,32.6,26.986957,5.613043,0
4762,2024,11,28,33.0,26.831461,6.168539,0
4763,2024,11,29,32.0,26.316484,5.683516,0


In [20]:
if os.path.exists(r"data\labelled\santacruz_labelled.csv"):
    print("Labelled processed data exists")
else:
    santacruz_df.to_csv(r"data\labelled\santacruz_labelled.csv", index= False)
    print("Labelled processed data created")

Labelled processed data created
